## Generate Text versions of Submission files

In [32]:
import os
import pypandoc
from pdf2docx  import parse

rootSubmissionFolder = os.path.join('data', 'Submissions')
textSubmissionsFolder = os.path.join('data', 'Converted Text Submissions')
tempFolder = 'temp'

for assignmentFolder in os.listdir(rootSubmissionFolder):
    assignmentFolderPath = os.path.join(rootSubmissionFolder, assignmentFolder)
    assignmentID = assignmentFolder.split('_')[-1]
    if not os.path.exists(os.path.join(textSubmissionsFolder, assignmentID)):
            os.mkdir(os.path.join(textSubmissionsFolder, assignmentID))

    for submissionFile in os.listdir(assignmentFolderPath):
        submissionFilePath = os.path.join(rootSubmissionFolder, assignmentFolder, submissionFile)
        fileFormat = submissionFile.split('.')[-1].lower()

        if 'LATE' in submissionFile:
            userID = submissionFile.split('_')[2]
        else:
            userID = submissionFile.split('_')[1]

        savedFileName = userID+'.txt'
        savedFilePath = os.path.join(textSubmissionsFolder, assignmentID, savedFileName)
        if os.path.exists(savedFilePath):
            continue
        print(submissionFilePath)
        
        try:
            if fileFormat=='docx':
                output = pypandoc.convert_file(submissionFilePath, 'plain')
            elif fileFormat=='pdf':
                tempFilePath = os.path.join(tempFolder, 'tempFile.docx')
                parse(submissionFilePath, tempFilePath)
                output = pypandoc.convert_file(tempFilePath, 'plain')
        
            if len(output.split('\n')) < 16:
                print('File seems to have no text content in it. Skipping file.')
            else:
                with open(savedFilePath, 'w') as textFile:
                    textFile.write(output)
        except Exception as e:
            print(f'Error in conversion: {e}')
            print('Skipping file.')
            continue
        


data\Submissions\submissions_First_Draft_-_Writing_Assignment_3_1884808\muckleroyjackgage_716559_30170642_Econ Essay 3-2.docx
Error in conversion: Pandoc died with exitcode "63" during conversion: couldn't unpack docx container: Did not find end of central directory signature

Skipping file.


[INFO] Start to convert data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\anwunahchukwubuikemalexander_729220_29770850_writing assignment 2.docx - Google Docs.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\anwunahchukwubuikemalexander_729220_29770850_writing assignment 2.docx - Google Docs.pdf


[WARNING] Ignore Line "writing assignment 2" due to overlap
[INFO] [3/4] Parsing pages...
[INFO] (1/1) Page 1
[INFO] [4/4] Creating pages...
[INFO] (1/1) Page 1
[INFO] Terminated in 3.68s.
[INFO] Start to convert data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\komindy_719136_29706023_Revised WA 2.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


File seems to have no text content in it. Skipping file.
data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\komindy_719136_29706023_Revised WA 2.pdf


...

File seems to have no text content in it. Skipping file.
data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\linzeyang_717688_29792724_Econ Writing 2 - Google Docs-1.pdf


[INFO] Terminated in 0.20s.
[INFO] Start to convert data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\pandaaryav_646518_29771902_Writing Assignment 2.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


File seems to have no text content in it. Skipping file.
data\Submissions\submissions_Revised_Draft_-_Writing_Assignment_2_1884818\pandaaryav_646518_29771902_Writing Assignment 2.pdf


[WARNING] Words count: 0. It might be a scanned pdf, which is not supported yet.
[INFO] [3/4] Parsing pages...
[INFO] (1/3) Page 1
[INFO] (2/3) Page 2
[INFO] (3/3) Page 3
[INFO] [4/4] Creating pages...
[INFO] (1/3) Page 1
[INFO] (2/3) Page 2
[INFO] (3/3) Page 3
[INFO] Terminated in 7.97s.


File seems to have no text content in it. Skipping file.


In [31]:
textSubmissionsFolder = os.path.join('data', 'Converted Text Submissions')
tempFolder = 'temp'
for assignmentID in os.listdir(textSubmissionsFolder):
    assignmentFolderPath = os.path.join(textSubmissionsFolder, assignmentID)

    for submissionFile in os.listdir(assignmentFolderPath):
        submissionFilePath = os.path.join(textSubmissionsFolder, assignmentID, submissionFile)
        with open(submissionFilePath) as textFile:
            submission = textFile.readlines()

        if len(submission) < 16:
            print(submissionFilePath)

## Token counting and checking

In [3]:
from helper import *
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str, encoding_name='cl100k_base') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

config = Config()
config.setFromEnv()

versionControl = 'V3'
promptVersion='P2'
courseName = 'ECON'
fullName = f'{courseName}-{versionControl}-{promptVersion}'
config.setSaveDetails(fullName)

config.overWriteSave = False

gradeRubricAssignmentDF = getGRAData(config)

for index, row in gradeRubricAssignmentDF.iterrows():
    fullPrompt = processTokenCount(row, config)
    if fullPrompt:
        tokenCount = num_tokens_from_string(fullPrompt)
        if tokenCount > 8192:
            display(row)


2023-07-03T12:59:31-0400 INFO [helper.py:411] - All configuration parameters set up successfully.
